In [ ]:
import numpy as np
import zlib
import base64
import xmltodict
import yt

In [ ]:
x = open("../datasets/aspect/solution-00100.0001.vtu", "rb").readlines()

In [ ]:
d = x[14]

In [ ]:
split_location = d.find(b"==") + 2

In [ ]:
split_location

In [ ]:
first = d[:split_location]
second = d[split_location:]

In [ ]:
type(second)

In [ ]:
a1 = np.frombuffer(base64.decodebytes(first), dtype="<f4")
a2 = np.frombuffer(zlib.decompress(base64.decodebytes(second)), dtype="<f4")

In [ ]:
print(len(a2))
print(a2.shape)

In [ ]:
print(a1)
print(a2.min(), a2.max(), a2.size, a2.size // 3)

In [ ]:
with open("../datasets/aspect/solution-00100.0001.vtu") as data:
    xml = xmltodict.parse(data.read())

In [ ]:
piece_id = 1

In [ ]:
xml['VTKFile']['UnstructuredGrid']['Piece'][piece_id]['PointData']['DataArray'][0]['@Name']
# coordinates = points data array
# cells data array = connectivity 
# for loop over xml to get array of conn, coords, offsets, types and a dict that maps field to vals

In [ ]:
xml['VTKFile']['UnstructuredGrid']['Piece'][piece_id]['PointData'];

In [ ]:
type(xml['VTKFile']['UnstructuredGrid']['Piece'][piece_id]['PointData']['DataArray'][0])

In [ ]:
import zlib

In [ ]:
def decode_binary(blob, use_zlib = True, dtype="<f4"):
    split_location = blob.find(b"==") + 2
    first = base64.decodebytes(blob[:split_location])
    second = base64.decodebytes(blob[split_location:])
    if zlib:
        second = zlib.decompress(second)
    return np.frombuffer(first, dtype="<f4"), np.frombuffer(second, dtype=dtype)

In [ ]:
def get_data(piece_id):
    data = {}
    for data_array in xml['VTKFile']['UnstructuredGrid']['Piece'][piece_id]['PointData']['DataArray']:
        # get names of arrays
        names = data_array['@Name']
        types = data_array['@type']
        # get binary code (as string)
        text = data_array['#text']
        # re-encode string
        etext = text.encode()
        metadata, data_field = decode_binary(etext)
        data[names] = data_field

    vel = data.pop("velocity")
    vel = vel.reshape((vel.size // 3, 3))
    data["velocity_x"] = vel[:,0]
    data["velocity_y"] = vel[:,1]
    data["velocity_z"] = vel[:,2]

    _, coords = decode_binary(xml['VTKFile']['UnstructuredGrid']['Piece'][piece_id]["Points"]["DataArray"]["#text"].encode())
    _, conn = decode_binary(xml['VTKFile']['UnstructuredGrid']['Piece'][piece_id]["Cells"]["DataArray"][0]["#text"].encode(),
                 dtype="u4")
    _, offsets = decode_binary(xml['VTKFile']['UnstructuredGrid']['Piece'][piece_id]["Cells"]["DataArray"][1]["#text"].encode(),
                 dtype="u4")
    _, cell_types = decode_binary(xml['VTKFile']['UnstructuredGrid']['Piece'][piece_id]["Cells"]["DataArray"][2]["#text"].encode(),
                 dtype="u1")
   
    coords = coords.reshape((coords.size // 3, 3))

    connectivity = conn.reshape((conn.size // 8, 8))
    node_data = {("connect1", _) : __[connectivity].astype("f8") for _, __ in data.items()}

    return coords, connectivity, node_data

In [ ]:
len(xml['VTKFile']['UnstructuredGrid']['Piece'])

In [ ]:
connectivity = []
coords = []
node_data = []
for i in range(len(xml['VTKFile']['UnstructuredGrid']['Piece'])):
    v1, v2, v3 = get_data(0)
    coords.append(v1)
    connectivity.append(v2)
    node_data.append(v3)
connectivity = np.concatenate(connectivity, axis=0)
coords = np.concatenate(coords, axis=0)
n = {}
for key in node_data[0]:
    n[key] = np.concatenate([_[key] for _ in node_data], axis=0)
node_data = n

ds = yt.load_unstructured_mesh(
    connectivity.astype("i8"),
    coords.astype("f8"),
    node_data = node_data
)

In [ ]:
ds.r[:]["viscosity"]

In [ ]:
yt.SlicePlot(ds, "y", ("connect1", "plastic_strain")).annotate_mesh_lines()

In [ ]:
s = yt.create_scene(ds, field=("connect1", "plastic_strain"), ord)

In [ ]:
s.camera.set_position((ds.domain_center[0], ds.domain_center[1], ds.domain_left_edge[2]))
s.set_camera(s.camera)

In [ ]:
s.show()
s.annotate_mesh_lines()
s.show()